In [1]:
!pip install -U pip
!pip install -U pandas
!pip install -U altair
!pip install -U sqlalchemy

In [2]:
import pandas as pd
import altair as alt
import urllib

base_url = 'https://ray.boxleft.com/cincy-tech-dev-survey/cincy-tech-survey'

In [3]:
sql = '''\
select
    case
    when years_exp = 'Still in school' then 'in-school'
    when years_exp = '0-1'             then ' 0 - 1'
    when years_exp = '2-4'             then ' 2 - 4'
    when years_exp = '4-6'             then ' 4 - 6'
    when years_exp = '6-8'             then ' 6 - 8'
    when years_exp = '8-10'            then ' 8 - 10'
    when years_exp = '10-15'           then '10 - 15'
    when years_exp = '15+'             then '>15'
    else '' || years_exp
  end as years_exp,
  happy_score,
  results_year,
  count(*) as count
from
  all_years_results_view
-- where
--   results_year = cast(:results_year as integer)
group by
  1,
  2,
  3
'''

query = urllib.parse.urlencode({'sql':sql,})

query_url = base_url + '.csv?' + query
print(query_url)

# create the dataframe from the Datasette query...
df = pd.read_csv(query_url)

df.head()

https://ray.boxleft.com/cincy-tech-dev-survey/cincy-tech-survey.csv?sql=select%0A++++case%0A++++when+years_exp+%3D+%27Still+in+school%27+then+%27in-school%27%0A++++when+years_exp+%3D+%270-1%27+++++++++++++then+%27+0+-+1%27%0A++++when+years_exp+%3D+%272-4%27+++++++++++++then+%27+2+-+4%27%0A++++when+years_exp+%3D+%274-6%27+++++++++++++then+%27+4+-+6%27%0A++++when+years_exp+%3D+%276-8%27+++++++++++++then+%27+6+-+8%27%0A++++when+years_exp+%3D+%278-10%27++++++++++++then+%27+8+-+10%27%0A++++when+years_exp+%3D+%2710-15%27+++++++++++then+%2710+-+15%27%0A++++when+years_exp+%3D+%2715%2B%27+++++++++++++then+%27%3E15%27%0A++++else+%27%27+%7C%7C+years_exp%0A++end+as+years_exp%2C%0A++happy_score%2C%0A++results_year%2C%0A++count%28%2A%29+as+count%0Afrom%0A++all_years_results_view%0A--+where%0A--+++results_year+%3D+cast%28%3Aresults_year+as+integer%29%0Agroup+by%0A++1%2C%0A++2%2C%0A++3%0A


,years_exp,happy_score,results_year,count
0,0 - 1,1.0,2019,1
1,0 - 1,2.0,2017,1
2,0 - 1,2.0,2021,1
3,0 - 1,3.0,2017,1
4,0 - 1,3.0,2019,2


In [4]:
# # plot source dataframe with Altair...
# alt.Chart(df).mark_bar(color='darkred').encode(
#     x = alt.X('count_total_items:Q', ),
#     y = alt.Y('item_format', sort='-x'),
#     tooltip=['item_format', 'count_total_items']
# ).properties(
#     title='Counts of Item by Item Format -- Current Collection'
# )

In [5]:
alt.Chart(df).mark_bar().encode(
    y=alt.X('years_exp:N'),
    x=alt.Y('count:Q', sort='-x'),
    color=('happy_score:O'),
    tooltip=['years_exp', 'happy_score', 'count'],
    facet=alt.Facet('results_year:O', columns=1),
).properties(
    title='Cincy Tech Survey: Happiness by Years Experience',
).configure_title(
    anchor='middle'
)

alt.Chart(...)

In [6]:
sql = '''\
with data as (
  select
    years_exp,
    happy_score,
    results_year,
    count(*) as count
  from
    all_years_results_view
  group by
    1,
    2,
    3
)
select
  (
    select
    round( (data.count * 1.0 / count(*) * 1.0) * 100.0, 2)
    from
    all_years_results_view as a
    where
    a.results_year = data.results_year
    and a.years_exp = data.years_exp
  ) as percent_total,
  case
    when years_exp = 'Still in school' then '  in-school'
    when years_exp = '0-1'             then ' 0 - 1'
    when years_exp = '2-4'             then ' 2 - 4'
    when years_exp = '4-6'             then ' 4 - 6'
    when years_exp = '6-8'             then ' 6 - 8'
    when years_exp = '8-10'            then ' 8 - 10'
    when years_exp = '10-15'           then '10 - 15'
    when years_exp = '15+'             then '>15'
    else '' || years_exp
  end as "years experience",
  happy_score as "happiness score",
  results_year as "results year",
  count
from
  data
  
order by
years_exp, results_year
'''

query = urllib.parse.urlencode({'sql':sql,})

query_url = base_url + '.csv?' + query
print(query_url)

# create the dataframe from the Datasette query...
df = pd.read_csv(query_url)

df.head()

https://ray.boxleft.com/cincy-tech-dev-survey/cincy-tech-survey.csv?sql=with+data+as+%28%0A++select%0A++++years_exp%2C%0A++++happy_score%2C%0A++++results_year%2C%0A++++count%28%2A%29+as+count%0A++from%0A++++all_years_results_view%0A++group+by%0A++++1%2C%0A++++2%2C%0A++++3%0A%29%0Aselect%0A++%28%0A++++select%0A++++round%28+%28data.count+%2A+1.0+%2F+count%28%2A%29+%2A+1.0%29+%2A+100.0%2C+2%29%0A++++from%0A++++all_years_results_view+as+a%0A++++where%0A++++a.results_year+%3D+data.results_year%0A++++and+a.years_exp+%3D+data.years_exp%0A++%29+as+percent_total%2C%0A++case%0A++++when+years_exp+%3D+%27Still+in+school%27+then+%27++in-school%27%0A++++when+years_exp+%3D+%270-1%27+++++++++++++then+%27+0+-+1%27%0A++++when+years_exp+%3D+%272-4%27+++++++++++++then+%27+2+-+4%27%0A++++when+years_exp+%3D+%274-6%27+++++++++++++then+%27+4+-+6%27%0A++++when+years_exp+%3D+%276-8%27+++++++++++++then+%27+6+-+8%27%0A++++when+years_exp+%3D+%278-10%27++++++++++++then+%27+8+-+10%27%0A++++when+years_exp+%3D+%2710-1

,percent_total,years experience,happiness score,results year,count
0,12.5,0 - 1,2.0,2017,1
1,12.5,0 - 1,3.0,2017,1
2,50.0,0 - 1,4.0,2017,4
3,25.0,0 - 1,5.0,2017,2
4,10.0,0 - 1,1.0,2019,1


In [7]:
chart = alt.Chart(df).mark_bar().encode(
    y=alt.X('years experience:N'),
    x=alt.Y('percent_total:Q', sort='-x', scale=alt.Scale(domain=(0, 100))),
    color=('happiness score:Q'),
    tooltip=['years experience', 'happiness score', 'percent_total', 'count'],
    facet=alt.Facet('results year:O', columns=1),
).properties(
    title='Cincy Tech Survey: Happiness by Years Experience',
).configure_title(
    anchor='middle'
)

chart

alt.Chart(...)

In [8]:
chart.save('CincyTechSurveyHappinessbyYearsExperience.html')

In [9]:
# Happiness by remote / in office status

sql = '''\
with data as (
  select
    happy_score,
    -- gender,
    remote,
    results_year,
    count(*) as count
  from
    all_years_results_view
  where
    remote in (
      'Work at a company in or around Cincinnati',
      'I work remotely'
    )
  group by
    1,
    2,
    3
)
select
  (
    select
      round((data.count * 1.0 / count(*) * 1.0) * 100.0, 2)
    from
      all_years_results_view as a
    where
      a.results_year = data.results_year
      and a.remote = data.remote
  ) as percent_total,
  happy_score as 'happiness score',
  case
    when remote = 'I work remotely' then 'Remote Work'
    when remote = 'Work at a company in or around Cincinnati' then 'On-Site / In-Office Work'
    else 'N/A'
  end as 'remote status',
  results_year as 'results year',
  count
from
  data
order by
  results_year,
  remote,
  happy_score
'''

query = urllib.parse.urlencode({'sql':sql,})

query_url = base_url + '.csv?' + query
print(query_url)

# create the dataframe from the Datasette query...
df = pd.read_csv(query_url)

df.head()

https://ray.boxleft.com/cincy-tech-dev-survey/cincy-tech-survey.csv?sql=with+data+as+%28%0A++select%0A++++happy_score%2C%0A++++--+gender%2C%0A++++remote%2C%0A++++results_year%2C%0A++++count%28%2A%29+as+count%0A++from%0A++++all_years_results_view%0A++where%0A++++remote+in+%28%0A++++++%27Work+at+a+company+in+or+around+Cincinnati%27%2C%0A++++++%27I+work+remotely%27%0A++++%29%0A++group+by%0A++++1%2C%0A++++2%2C%0A++++3%0A%29%0Aselect%0A++%28%0A++++select%0A++++++round%28%28data.count+%2A+1.0+%2F+count%28%2A%29+%2A+1.0%29+%2A+100.0%2C+2%29%0A++++from%0A++++++all_years_results_view+as+a%0A++++where%0A++++++a.results_year+%3D+data.results_year%0A++++++and+a.remote+%3D+data.remote%0A++%29+as+percent_total%2C%0A++happy_score+as+%27happiness+score%27%2C%0A++case%0A++++when+remote+%3D+%27I+work+remotely%27+then+%27Remote+Work%27%0A++++when+remote+%3D+%27Work+at+a+company+in+or+around+Cincinnati%27+then+%27On-Site+%2F+In-Office+Work%27%0A++++else+%27N%2FA%27%0A++end+as+%27remote+status%27%2C%0A++re

,percent_total,happiness score,remote status,results year,count
0,8.33,2.0,Remote Work,2017,1
1,25.00,3.0,Remote Work,2017,3
2,16.67,4.0,Remote Work,2017,2
3,50.00,5.0,Remote Work,2017,6
4,1.47,1.0,On-Site / In-Office Work,2017,1


In [10]:
chart = alt.Chart(df).mark_bar().encode(
    y=alt.X('remote status:N', title=None), 
    x=alt.Y('percent_total:Q', sort='-x', scale=alt.Scale(domain=(0, 100))),
    color=('happiness score:Q'),
    tooltip=['results year', 'remote status', 'happiness score', 'percent_total', 'count'],
    facet=alt.Facet('results year:O', columns=1),
).properties(
    title='Cincy Tech Survey: Happiness by Remote Status',
).configure_title(
    anchor='middle'
)

chart

alt.Chart(...)

In [11]:
chart.save('CincyTechSurveyHappinessbyRemoteStatus.html')

In [12]:
'''
The 10 Most Popular Programming Languages to Learn in 2021

https://www.northeastern.edu/graduate/blog/most-popular-programming-languages/
'''

language_rank_2020 = [
    {
        'northeastern_rank':  1,
        'language_name': '"Python"',
        'indeed_avg_salary':  120000,
        'indeed_link': 'https://www.indeed.com/career/python-developer/salaries'
    },
    {
        'northeastern_rank':  2,
        'language_name': '"JavaScript"',
        'indeed_avg_salary':  118000,
        'indeed_link': 'https://www.indeed.com/career/javascript-developer/salaries'
    },
    {
        'northeastern_rank':  3,
        'language_name': '"Java"',
        'indeed_avg_salary':  104000,
        'indeed_link': 'https://www.indeed.com/career/java-developer/salaries'
    },
    {
        'northeastern_rank':  4,
        'language_name': '"C#" OR ".net" OR "C Sharp"',
        # also known as .NET
        'indeed_avg_salary':  97000,
        'indeed_link': 'https://www.indeed.com/career/.net-developer/salaries'
    },
    {
        'northeastern_rank':  5,
        'language_name': '"C"',
        'indeed_avg_salary':  99652,
        'indeed_link': 'https://www.indeed.com/career/c-developer/salaries'
    },
    {
        'northeastern_rank':  6,
        'language_name': '"C++"',
        'indeed_avg_salary':  113915,
        'indeed_link': 'https://www.indeed.com/career/c%2B%2B-developer/salaries'
    },
    {
        'northeastern_rank':  7,
        'language_name': '"Go"',
        'indeed_avg_salary':  93000,
        'indeed_link': None
    },
    {
        'northeastern_rank':  8,
        'language_name': '"R"',
        'indeed_avg_salary':  93000,
        'indeed_link': None
    },
    {
        'northeastern_rank':  9,
        'language_name': '"Swift"',
        'indeed_avg_salary':  93000,
        'indeed_link': None
    },
    {
        'northeastern_rank':  10,
        'language_name': '"PHP"',
        'indeed_avg_salary':  81000,
        'indeed_link': None
    },
    
]

In [13]:
# language search ... supply {language_name} and {results_year}
sql = '''
with data as (
  select
    '{results_year}' as results_year,
    '{language_name}' as language_name,
    cast(results_{results_year}.salary_norm as numeric) as salary_norm
  from
    results_{results_year}_fts
    join results_{results_year} on results_{results_year}.rowid = results_{results_year}_fts.rowid
  where
    results_{results_year}_fts match '{language_name}'
    and cast(results_{results_year}.salary_norm as numeric) > 0
)
select
  results_year,
  language_name,
  count(*) as count,
  max(salary_norm) as max_salary,
  min(salary_norm) as min_salary,
  (
    select
      cast((avg(salary_norm)) as integer)
    from
      data
    where
      salary_norm > 0
  ) as avg_salary,
  (
    SELECT
      cast((avg(salary_norm)) as integer)
    FROM
      (
        SELECT
          salary_norm
        FROM
          data
        ORDER BY
          salary_norm
        LIMIT
          2 OFFSET (
            SELECT
              (COUNT(*) - 1) / 2
            FROM
              data
          )
      )
  ) as median_salary
from
  data
'''

In [14]:
# language_rank_2020[0]['language_name']
for i, data in enumerate(language_rank_2020):
    print(i, data)
    # print(sql.format(language_name = data['language_name'], results_year=2017))

    df = pd.DataFrame()
    
    query = urllib.parse.urlencode({'sql':sql.format(language_name = data['language_name'], results_year=2021)})
    query_url = base_url + '.csv?' + query
    # print(query_url)
    # create the dataframe from the Datasette query...
    df = df.append(pd.read_csv(query_url), ignore_index=True)
    
    query = urllib.parse.urlencode({'sql':sql.format(language_name = data['language_name'], results_year=2019)})
    query_url = base_url + '.csv?' + query
    # print(query_url)
    # create the dataframe from the Datasette query...
    df = df.append(pd.read_csv(query_url), ignore_index=True)
    
    query = urllib.parse.urlencode({'sql':sql.format(language_name = data['language_name'], results_year=2017)})
    query_url = base_url + '.csv?' + query
    # print(query_url)
    # create the dataframe from the Datasette query...
    df = df.append(pd.read_csv(query_url), ignore_index=True)

    print(df.head())
    
    if i == 5:
        break

    
# 'northeastern_rank':  10,
#         'language_name': 'PHP',
#         'indeed_avg_salary':  81000,
#         'indeed_link': None

0 {'northeastern_rank': 1, 'language_name': '"Python"', 'indeed_avg_salary': 120000, 'indeed_link': 'https://www.indeed.com/career/python-developer/salaries'}
   results_year language_name  count  max_salary  min_salary  avg_salary  \
0          2021      "Python"     24    320000.0     40500.0      113723   
1          2019      "Python"     11    120000.0     50000.0       88909   
2          2017      "Python"     14    120000.0     24000.0       83132   

   median_salary  
0         104687  
1          87000  
2          87500  
1 {'northeastern_rank': 2, 'language_name': '"JavaScript"', 'indeed_avg_salary': 118000, 'indeed_link': 'https://www.indeed.com/career/javascript-developer/salaries'}
   results_year language_name  count  max_salary  min_salary  avg_salary  \
0          2021  "JavaScript"     35    180000.0     45000.0      109996   
1          2019  "JavaScript"     37    175000.0     16000.0       94267   
2          2017  "JavaScript"     42    177395.0     24000.0     

In [15]:
df.head()

,results_year,language_name,count,max_salary,min_salary,avg_salary,median_salary
0,2021,"""C++""",20,320000.0,65000.0,114443,103687
1,2019,"""C++""",20,167000.0,16000.0,98185,96000
2,2017,"""C++""",19,135000.0,22500.0,77536,79000


In [16]:
query_url

'https://ray.boxleft.com/cincy-tech-dev-survey/cincy-tech-survey.csv?sql=%0Awith+data+as+%28%0A++select%0A++++%272017%27+as+results_year%2C%0A++++%27%22C%2B%2B%22%27+as+language_name%2C%0A++++cast%28results_2017.salary_norm+as+numeric%29+as+salary_norm%0A++from%0A++++results_2017_fts%0A++++join+results_2017+on+results_2017.rowid+%3D+results_2017_fts.rowid%0A++where%0A++++results_2017_fts+match+%27%22C%2B%2B%22%27%0A++++and+cast%28results_2017.salary_norm+as+numeric%29+%3E+0%0A%29%0Aselect%0A++results_year%2C%0A++language_name%2C%0A++count%28%2A%29+as+count%2C%0A++max%28salary_norm%29+as+max_salary%2C%0A++min%28salary_norm%29+as+min_salary%2C%0A++%28%0A++++select%0A++++++cast%28%28avg%28salary_norm%29%29+as+integer%29%0A++++from%0A++++++data%0A++++where%0A++++++salary_norm+%3E+0%0A++%29+as+avg_salary%2C%0A++%28%0A++++SELECT%0A++++++cast%28%28avg%28salary_norm%29%29+as+integer%29%0A++++FROM%0A++++++%28%0A++++++++SELECT%0A++++++++++salary_norm%0A++++++++FROM%0A++++++++++data%0A++++++++ORD

# salary by experience

look at the distribution of salary values to get some good bin sizes ...


In [17]:
import locale
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )

# use like this
# locale.currency( 188518982.18, grouping=True )
# '$188,518,982.18'

'en_US.UTF-8'

In [18]:
sql = '''\
with data as (
  select
    years_exp,
    cast(salary_norm as numeric) as salary_norm,
    results_year
  from
    all_years_results_view
  where
    cast(salary_norm as numeric) > 0
)
select
  case
    when years_exp = 'Still in school' then '  in-school'
    when years_exp = '0-1' then ' 0 - 1'
    when years_exp = '2-4' then ' 2 - 4'
    when years_exp = '4-6' then ' 4 - 6'
    when years_exp = '6-8' then ' 6 - 8'
    when years_exp = '8-10' then ' 8 - 10'
    when years_exp = '10-15' then '10 - 15'
    when years_exp = '15+' then '>15'
    else '' || years_exp
  end as "years experience",
  salary_norm,
  results_year as "results year"
from
  data
order by
  years_exp,
  results_year
'''

query = urllib.parse.urlencode({'sql':sql.format(language_name = data['language_name'], results_year=2021)})
query_url = base_url + '.csv?' + query
    # print(query_url)
    # create the dataframe from the Datasette query...
df = pd.read_csv(query_url)

df.head()

,years experience,salary_norm,results year
0,0 - 1,49000.0,2017
1,0 - 1,59000.0,2017
2,0 - 1,50000.0,2017
3,0 - 1,51000.0,2017
4,0 - 1,65000.0,2017


In [19]:
# these are the approximate bins
q=[0.2, 0.4, 0.6, 0.8]
v=[]
for i, value in enumerate(q):
    print(round(df['salary_norm'].quantile(value)))
    v.append(round(df['salary_norm'].quantile(value)))

70000
90000
118800
135000


In [20]:
print(df[(df['salary_norm'] > 0.0) & (df['salary_norm'] <= v[0])].shape)
print(df[(df['salary_norm'] > v[0]) & (df['salary_norm'] <= v[1])].shape)
print(df[(df['salary_norm'] > v[1]) & (df['salary_norm'] <= v[2])].shape)
print(df[(df['salary_norm'] > v[2]) & (df['salary_norm'] <= v[3])].shape)
print(df[(df['salary_norm'] > v[3])].shape)

(60, 3)
(57, 3)
(52, 3)
(59, 3)
(54, 3)


In [21]:
sql = '''\
with data as (
  select
    years_exp,
    cast(salary_norm as numeric) as salary_norm,
    results_year
  from
    all_years_results_view
  where
    cast(salary_norm as numeric) > 0
)
select
  case
    when years_exp = 'Still in school' then '  in-school'
    when years_exp = '0-1' then ' 0 - 1'
    when years_exp = '2-4' then ' 2 - 4'
    when years_exp = '4-6' then ' 4 - 6'
    when years_exp = '6-8' then ' 6 - 8'
    when years_exp = '8-10' then ' 8 - 10'
    when years_exp = '10-15' then '10 - 15'
    when years_exp = '15+' then '>15'
    else '' || years_exp
  end as "years experience",
  case
  when salary_norm > 0.0 and salary_norm <= {range_0} then '   <= {range_0_format}'
  when salary_norm > {range_0} and salary_norm <= {range_1} then '  {range_0_format} to {range_1_format}'
  when salary_norm > {range_1} and salary_norm <= {range_2} then '  {range_1_format} to {range_2_format}'
  when salary_norm > {range_2} and salary_norm <= {range_3} then '{range_2_format} to {range_3_format}'
  when salary_norm > {range_3} then '> {range_3_format}'
  end as salary_range,
  results_year as "results year",
  count(*) as count
from
  data
group by
  1,2,3
'''.format(
    range_0=v[0], range_1=v[1], range_2=v[2], range_3=v[3],
    range_0_format=locale.format_string('$%d', v[0], grouping=True),
    range_1_format=locale.format_string('$%d', v[1], grouping=True),
    range_2_format=locale.format_string('$%d', v[2], grouping=True),
    range_3_format=locale.format_string('$%d', v[3], grouping=True),
)

# print(sql)

query = urllib.parse.urlencode({'sql':sql})
query_url = base_url + '.csv?' + query
#     # print(query_url)
#     # create the dataframe from the Datasette query...
df = pd.read_csv(query_url)

print(df.head())

  years experience   salary_range  results year  count
0        in-school     <= $70,000          2017      1
1        in-school     <= $70,000          2019      2
2            0 - 1     <= $70,000          2017      7
3            0 - 1     <= $70,000          2019      7
4            0 - 1     <= $70,000          2021      7


In [22]:
chart = alt.Chart(df).mark_bar().encode(    
    x=alt.X('count:Q', sort='-y'),
    y=alt.Y('years experience',),
    tooltip=['results year', 'years experience', 'salary_range', 'count'],
    color='salary_range:O',
    facet=alt.Facet('results year:O', columns=1),
).properties(
    title='Cincy Tech Survey: Salary by Years Experience'
).configure_title(
    # fontSize=20,
    anchor='middle',
)

chart

alt.Chart(...)

In [23]:
chart.save('CincyTechSurveyCountSalaryRangeByYearsExp.html')

In [24]:
from sqlalchemy import create_engine

In [27]:
mem_engine = create_engine('sqlite:///')
df.to_sql('data', con=mem_engine)

data_df = pd.read_sql(con=mem_engine, sql='''\
select
data.'years experience',
data.'salary_range',
data.'results year',
count,
(
    select
      round((data.count * 1.0 / sum(count) * 1.0) * 100 , 2)
      
    from
      data as a
    where
      a.'results year' = data.'results year'
      and a.'years experience' = data.'years experience'
  ) as percent_total
from data
order by data.'results year', data.'years experience'
''')

# print(data_df.head)
data_df.head

<bound method NDFrame.head of    years experience           salary_range  results year  count  percent_total
0         in-school             <= $70,000          2017      1         100.00
1             0 - 1             <= $70,000          2017      7         100.00
2             2 - 4             <= $70,000          2017     12          66.67
3             2 - 4     $70,000 to $90,000          2017      5          27.78
4             2 - 4    $90,000 to $118,800          2017      1           5.56
..              ...                    ...           ...    ...            ...
73          10 - 15             > $135,000          2021      6          27.27
74              >15     $70,000 to $90,000          2021      1           3.85
75              >15    $90,000 to $118,800          2021      2           7.69
76              >15   $118,800 to $135,000          2021      7          26.92
77              >15             > $135,000          2021     16          61.54

[78 rows x 5 columns]

In [28]:
# produce the links for the chart
sql = '''\
select
  *
from
  results_{results_year}
where
  years_exp = '{years_exp}'
order by
  cast(salary_norm as numeric) DESC
'''

In [29]:
def gen_url(results_year, years_exp):
    if years_exp == '  in-school':
        exp = 'Still in school'
    if years_exp == ' 0 - 1':
        exp = '0-1'
    if years_exp == ' 2 - 4':
        exp = '2-4'
    if years_exp == ' 4 - 6':
        exp = '4-6'
    if years_exp == ' 6 - 8':
        exp = '6-8'
    if years_exp == ' 8 - 10':
        exp = '8-10'
    if years_exp == '10 - 15':
        exp = '10-15'
    if years_exp == '>15':
        exp = '15+'
    
    query = urllib.parse.urlencode({'sql':sql.format(results_year=results_year, years_exp=exp)})
    query_url = base_url + '?' + query
    return(query_url)

In [30]:
# test the function ... at least a little
i = 10
print(data_df.iloc[i]['results year'], data_df.iloc[i]['years experience'])
print(gen_url(results_year=data_df.iloc[i]['results year'], years_exp=data_df.iloc[i]['years experience'], ))

i = 40
print(data_df.iloc[i]['results year'], data_df.iloc[i]['years experience'])
print(gen_url(results_year=data_df.iloc[i]['results year'], years_exp=data_df.iloc[i]['years experience'], ))

2017  6 - 8
https://ray.boxleft.com/cincy-tech-dev-survey/cincy-tech-survey?sql=select%0A++%2A%0Afrom%0A++results_2017%0Awhere%0A++years_exp+%3D+%276-8%27%0Aorder+by%0A++cast%28salary_norm+as+numeric%29+DESC%0A
2019  8 - 10
https://ray.boxleft.com/cincy-tech-dev-survey/cincy-tech-survey?sql=select%0A++%2A%0Afrom%0A++results_2019%0Awhere%0A++years_exp+%3D+%278-10%27%0Aorder+by%0A++cast%28salary_norm+as+numeric%29+DESC%0A


In [31]:
data_df['url'] = data_df.apply(lambda x: gen_url(results_year=x['results year'], years_exp=x['years experience'], ), axis=1)

In [32]:
data_df['url']

0     https://ray.boxleft.com/cincy-tech-dev-survey/...
1     https://ray.boxleft.com/cincy-tech-dev-survey/...
2     https://ray.boxleft.com/cincy-tech-dev-survey/...
3     https://ray.boxleft.com/cincy-tech-dev-survey/...
4     https://ray.boxleft.com/cincy-tech-dev-survey/...
                            ...                        
73    https://ray.boxleft.com/cincy-tech-dev-survey/...
74    https://ray.boxleft.com/cincy-tech-dev-survey/...
75    https://ray.boxleft.com/cincy-tech-dev-survey/...
76    https://ray.boxleft.com/cincy-tech-dev-survey/...
77    https://ray.boxleft.com/cincy-tech-dev-survey/...
Name: url, Length: 78, dtype: object

In [33]:
chart = alt.Chart(data_df).mark_bar().encode(
    y=alt.Y('years experience:N'),
    x=alt.X('percent_total:Q', sort='-x', scale=alt.Scale(domain=(0, 100))),
    color=('salary_range:O'),
    tooltip=['results year', 'years experience', 'salary_range', 'percent_total' , 'count'],
    facet=alt.Facet('results year:O', columns=1),
    href='url'
).properties(
    title='Cincy Tech Survey: Salary by Years Experience',
).configure_title(
    anchor='middle'
)

chart

alt.Chart(...)

In [34]:
chart.save('CincyTechSurveySalaryRangeByYearsExp.html')

# salary ranges by gender

In [38]:
sql = '''\
with data as (
  select
    case
    when gender = 'Male' then ' Male'
    when gender = 'Female' then '  Female'
    else 'NA / Other'
  end as gender,
    cast(salary_norm as numeric) as salary_norm,
    results_year
  from
    all_years_results_view
  where
    cast(salary_norm as numeric) > 0
)
select
  gender,
  case
  when salary_norm > 0.0 and salary_norm <= {range_0} then '   <= {range_0_format}'
  when salary_norm > {range_0} and salary_norm <= {range_1} then '  {range_0_format} to {range_1_format}'
  when salary_norm > {range_1} and salary_norm <= {range_2} then '  {range_1_format} to {range_2_format}'
  when salary_norm > {range_2} and salary_norm <= {range_3} then '{range_2_format} to {range_3_format}'
  when salary_norm > {range_3} then '> {range_3_format}'
  end as salary_range,
  results_year as "results year",
  count(*) as count
from
  data
group by
  1,2,3
'''.format(
    range_0=v[0], range_1=v[1], range_2=v[2], range_3=v[3],
    range_0_format=locale.format_string('$%d', v[0], grouping=True),
    range_1_format=locale.format_string('$%d', v[1], grouping=True),
    range_2_format=locale.format_string('$%d', v[2], grouping=True),
    range_3_format=locale.format_string('$%d', v[3], grouping=True),
)

print(sql)

query = urllib.parse.urlencode({'sql':sql})
query_url = base_url + '.csv?' + query
#     # print(query_url)
#     # create the dataframe from the Datasette query...
df = pd.read_csv(query_url)

print(df.head())

with data as (
  select
    case
    when gender = 'Male' then ' Male'
    when gender = 'Female' then '  Female'
    else 'NA / Other'
  end as gender,
    cast(salary_norm as numeric) as salary_norm,
    results_year
  from
    all_years_results_view
  where
    cast(salary_norm as numeric) > 0
)
select
  gender,
  case
  when salary_norm > 0.0 and salary_norm <= 70000 then '   <= $70,000'
  when salary_norm > 70000 and salary_norm <= 90000 then '  $70,000 to $90,000'
  when salary_norm > 90000 and salary_norm <= 118800 then '  $90,000 to $118,800'
  when salary_norm > 118800 and salary_norm <= 135000 then '$118,800 to $135,000'
  when salary_norm > 135000 then '> $135,000'
  end as salary_range,
  results_year as "results year",
  count(*) as count
from
  data
group by
  1,2,3

     gender          salary_range  results year  count
0    Female            <= $70,000          2017      6
1    Female            <= $70,000          2019      1
2    Female            <= $70,000          

In [39]:
chart = alt.Chart(df).mark_bar().encode(    
    x=alt.X('count:Q', sort='-y'),
    y=alt.Y('gender',),
    tooltip=['results year', 'gender', 'salary_range', 'count'],
    color='salary_range:O',
    facet=alt.Facet('results year:O', columns=1),
).properties(
    title='Cincy Tech Survey: Salary by Gender'
).configure_title(
    # fontSize=20,
    anchor='middle',
)

chart

alt.Chart(...)

In [41]:
mem_engine = create_engine('sqlite:///')
df.to_sql('data', con=mem_engine)

data_df = pd.read_sql(con=mem_engine, sql='''\
select
data.'gender',
data.'salary_range',
data.'results year',
count,
(
    select
      round((data.count * 1.0 / sum(count) * 1.0) * 100 , 2)
      
    from
      data as a
    where
      a.'results year' = data.'results year'
      and a.'gender' = data.'gender'
  ) as percent_total
from data
order by data.'results year', data.'gender'
''')

# print(data_df.head)
data_df.head

<bound method NDFrame.head of         gender           salary_range  results year  count  percent_total
0       Female             <= $70,000          2017      6          50.00
1       Female     $70,000 to $90,000          2017      5          41.67
2       Female    $90,000 to $118,800          2017      1           8.33
3         Male             <= $70,000          2017     23          33.82
4         Male     $70,000 to $90,000          2017     15          22.06
5         Male    $90,000 to $118,800          2017     14          20.59
6         Male   $118,800 to $135,000          2017     12          17.65
7         Male             > $135,000          2017      4           5.88
8       Female             <= $70,000          2019      1          14.29
9       Female     $70,000 to $90,000          2019      5          71.43
10      Female    $90,000 to $118,800          2019      1          14.29
11        Male             <= $70,000          2019     16          22.22
12      

In [45]:
chart = alt.Chart(data_df).mark_bar().encode(
    y=alt.Y('gender:N', title=None),
    x=alt.X('percent_total:Q', sort='-x', scale=alt.Scale(domain=(0, 100))),
    color=('salary_range:O'),
    tooltip=['results year', 'gender', 'salary_range', 'percent_total' , 'count'],
    facet=alt.Facet('results year:O', columns=1),
    # href='url'
).properties(
    title='Cincy Tech Survey: Salary by Gender',
).configure_title(
    anchor='middle'
)

chart

alt.Chart(...)

In [46]:
chart.save('CincyTechSurveySalaryRangeByGender.html')